In [2]:
import os
from dotenv import load_dotenv


load_dotenv()


True

In [8]:
key = os.getenv("NVIDIA_LLAMA2_KEY")

if nvidia_key:
    print(f"NVIDIA_LLAMA2_KEY loaded: {key}")
else:
    print("NVIDIA_LLAMA2_KEY is not found or failed to load.")

NVIDIA_LLAMA2_KEY loaded: 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


In [5]:
import os
import json
import pandas as pd
import traceback
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2
from langchain_nvidia_ai_endpoints import ChatNVIDIA

In [13]:

llm = ChatNVIDIA(
  model="meta/llama2-70b",
  api_key=key, 
  temperature=0.45
)

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
TEMPLATE = """
You are an accomplished educational assessment designer specializing in creating engaging multiple-choice questions (MCQs). 

INPUT TEXT:
{text}

TASK:
Create {number} thought-provoking MCQs based on the provided text, tailored for a {tone} difficulty level.

REQUIREMENTS:
1. Subject Analysis:
   - Identify the primary subject area and specific topics covered
   - Consider the learning objectives evident in the text
   - Align questions with educational standards when applicable

2. Question Design:
   - Create questions that test different cognitive levels (recall, understanding, application, analysis)
   - Include a mix of:
     * Direct concept questions
     * Scenario-based questions
     * Application-oriented questions
     * Critical thinking questions
   - Ensure questions progress from simpler to more complex concepts
   - Avoid repetitive content or question patterns

3. Answer Options:
   - Provide 4 options per question
   - Include one clearly correct answer
   - Create plausible distractors that:
     * Reflect common misconceptions
     * Are grammatically consistent
     * Are similar in length
     * Follow logical ordering (numerical/alphabetical when applicable)

4. Quality Control:
   - Verify all questions are answerable from the provided text
   - Ensure language clarity and accessibility
   - Double-check factual accuracy
   - Avoid cultural or regional bias
   - Include explanations for correct answers
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "tone", "response_json"],
    template=TEMPLATE
    )

In [14]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_10944\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [16]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["quiz"], template=TEMPLATE)

In [17]:

review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [18]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "tone", "response_json"], 
                                        output_variables=["quiz", "review"], verbose=True,)

In [20]:
with open(r"F:\MCQ_Generator\data.txt", 'r') as file:
    TEXT = file.read()

In [21]:
print(TEXT)

Maharana Pratap (1540-1597) was a renowned Rajput ruler of Mewar (in present-day Rajasthan, India), celebrated for his fierce resistance against the Mughal Empire. He was the eldest son of Udai Singh II and succeeded to the throne of Mewar in 1572.
Most famously, Pratap refused to submit to Mughal Emperor Akbar, unlike many other Rajput rulers of his time. The Battle of Haldighati (1576) against Akbar's forces, led by Man Singh I, is particularly legendary. Though Pratap's forces were outnumbered, he fought valiantly. His loyal horse Chetak died protecting him in this battle.
Despite losing Chittor, his capital, Pratap continued guerrilla warfare from the hills of Aravalli. He lived in difficult conditions in the forests but never gave up his struggle for independence. He managed to reclaim much of Mewar except Chittor through strategic warfare and maintained his sovereignty.
Maharana Pratap is remembered as a symbol of Rajput valor, patriotism, and resistance against foreign rule. His

In [22]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [24]:
NUMBER=5 
TONE="Hard"

In [25]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\lenovo\AppData\Local\Temp\ipykernel_10944\2092646909.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an accomplished educational assessment designer specializing in creating engaging multiple-choice questions (MCQs). 

INPUT TEXT:
Maharana Pratap (1540-1597) was a renowned Rajput ruler of Mewar (in present-day Rajasthan, India), celebrated for his fierce resistance against the Mughal Empire. He was the eldest son of Udai Singh II and succeeded to the throne of Mewar in 1572.
Most famously, Pratap refused to submit to Mughal Emperor Akbar, unlike many other Rajput rulers of his time. The Battle of Haldighati (1576) against Akbar's forces, led by Man Singh I, is particularly legendary. Though Pratap's forces were outnumbered, he fought valiantly. His loyal horse Chetak died protecting him in this battle.
Despite losing Chittor, his capital, Pratap continued guerrilla warfare from the hills of Aravalli. He lived in difficult conditions in the forests but never gave up his strugg

In [26]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2882
Prompt Tokens:1720
Completion Tokens:1162
Total Cost:0.0


In [31]:
response

{'text': "Maharana Pratap (1540-1597) was a renowned Rajput ruler of Mewar (in present-day Rajasthan, India), celebrated for his fierce resistance against the Mughal Empire. He was the eldest son of Udai Singh II and succeeded to the throne of Mewar in 1572.\nMost famously, Pratap refused to submit to Mughal Emperor Akbar, unlike many other Rajput rulers of his time. The Battle of Haldighati (1576) against Akbar's forces, led by Man Singh I, is particularly legendary. Though Pratap's forces were outnumbered, he fought valiantly. His loyal horse Chetak died protecting him in this battle.\nDespite losing Chittor, his capital, Pratap continued guerrilla warfare from the hills of Aravalli. He lived in difficult conditions in the forests but never gave up his struggle for independence. He managed to reclaim much of Mewar except Chittor through strategic warfare and maintained his sovereignty.\nMaharana Pratap is remembered as a symbol of Rajput valor, patriotism, and resistance against fore

In [30]:
quiz=response.get("quiz")
print(quiz)

  Sure, here are 5 thought-provoking MCQs based on the provided text, tailored for a Hard difficulty level:

1. What was the primary reason for Maharana Pratap's resistance against the Mughal Empire?

A) He wanted to expand his kingdom and conquer new territories.
B) He was opposed to the Mughal Empire's religious policies.
C) He wanted to protect his homeland's independence and sovereignty.
D) He was angry with the Mughal Emperor Akbar's refusal to grant him a higher title.

Correct answer: C) He wanted to protect his homeland's independence and sovereignty.

2. Which battle did Maharana Pratap fight against the Mughal Empire, led by Man Singh I?

A) The Battle of Haldighati
B) The Battle of Chittor
C) The Battle of Aravalli
D) The Battle of Mewar

Correct answer: A) The Battle of Haldighati

3. What was the name of Maharana Pratap's loyal horse that died protecting him in the Battle of Haldighati?

A) Chetak
B) Raju
C) Raja
D) Ranveer

Correct answer: A) Chetak

4. How did Maharana P

In [32]:
import json
import re

def convert_to_json_format(input_text):
    """
    Convert the MCQ response into a JSON-compatible format.
    """
    # Split the input text into individual questions
    questions = re.split(r'\n\d+\.\s', input_text.strip())
    response_json = {}

    for i, question in enumerate(questions[1:], 1):  # Skip the first split part (intro text)
        # Extract question text
        mcq_match = re.match(r"(.*?)\n", question)
        mcq = mcq_match.group(1).strip() if mcq_match else ""

        # Extract options
        options_match = re.findall(r"([A-D])\)\s(.*?)(?=\n|$)", question)
        options = {opt[0].lower(): opt[1].strip() for opt in options_match}

        # Extract correct answer
        correct_match = re.search(r"Correct answer:\s([A-D])", question)
        correct = correct_match.group(1).lower() if correct_match else ""

        # Add to response_json
        response_json[str(i)] = {
            "mcq": mcq,
            "options": options,
            "correct": correct,
        }

    return json.dumps(response_json, indent=4)

# Convert to JSON
converted_json = convert_to_json_format(quiz)
print(converted_json)


{
    "1": {
        "mcq": "What was the primary reason for Maharana Pratap's resistance against the Mughal Empire?",
        "options": {
            "a": "He wanted to expand his kingdom and conquer new territories.",
            "b": "He was opposed to the Mughal Empire's religious policies.",
            "c": "He wanted to protect his homeland's independence and sovereignty.",
            "d": "He was angry with the Mughal Emperor Akbar's refusal to grant him a higher title."
        },
        "correct": "c"
    },
    "2": {
        "mcq": "Which battle did Maharana Pratap fight against the Mughal Empire, led by Man Singh I?",
        "options": {
            "a": "The Battle of Haldighati",
            "b": "The Battle of Chittor",
            "c": "The Battle of Aravalli",
            "d": "The Battle of Mewar"
        },
        "correct": "a"
    },
    "3": {
        "mcq": "What was the name of Maharana Pratap's loyal horse that died protecting him in the Battle of Haldigh

In [34]:
quiz=json.loads(converted_json)

In [38]:
import pandas as pd
from typing import Dict, List
import logging
from datetime import datetime

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [39]:
class QuizProcessor:
    """
    A class to process quiz data with options split into separate columns
    """
    
    def __init__(self, quiz_data: Dict):
        self.quiz = quiz_data
        self.processed_data = []
        self.logger = logging.getLogger(__name__)

    def process_quiz_data(self) -> List[Dict]:
        """Process quiz data with separate columns for each option"""
        self.logger.info("Starting quiz data processing...")
        
        try:
            for question_id, question_data in self.quiz.items():
                processed_question = {
                    "Question": question_data["mcq"],
                    "Option_A": question_data["options"]["a"],
                    "Option_B": question_data["options"]["b"],
                    "Option_C": question_data["options"]["c"],
                    "Option_D": question_data["options"]["d"],
                    "Correct_Answer": question_data["correct"].upper()
                }
                self.processed_data.append(processed_question)
                
            self.logger.info(f"Successfully processed {len(self.processed_data)} questions")
            return self.processed_data
            
        except Exception as e:
            self.logger.error(f"Error processing quiz data: {e}")
            raise

In [40]:
# Create processor instance and process data
processor = QuizProcessor(quiz)
processed_data = processor.process_quiz_data()

# Convert to DataFrame
quiz_df = pd.DataFrame(processed_data)

# Display the DataFrame with better formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print("\nQuiz DataFrame with Split Options:")
quiz_df

2024-11-18 02:22:17,136 - INFO - Starting quiz data processing...
2024-11-18 02:22:17,138 - INFO - Successfully processed 5 questions



Quiz DataFrame with Split Options:


,Question,Option_A,Option_B,Option_C,Option_D,Correct_Answer
0,What was the primary reason for Maharana Prata...,He wanted to expand his kingdom and conquer ne...,He was opposed to the Mughal Empire's religiou...,He wanted to protect his homeland's independen...,He was angry with the Mughal Emperor Akbar's r...,C
1,Which battle did Maharana Pratap fight against...,The Battle of Haldighati,The Battle of Chittor,The Battle of Aravalli,The Battle of Mewar,A
2,What was the name of Maharana Pratap's loyal h...,Chetak,Raju,Raja,Ranveer,A
3,How did Maharana Pratap manage to reclaim much...,By forming alliances with neighboring kingdoms,By using guerrilla warfare tactics,By paying a large ransom to the Mughal Empire,By surrendering to the Mughal Empire,B
4,Which of the following best describes Maharana...,Autocratic,Democratic,Laissez-faire,Transformational,D


In [42]:
try:
    # Export to CSV
    quiz_df.to_csv("MCQ_generated.csv", index=False)
    print(f"\nSuccessfully exported quiz data to machinelearning_split_options.csv")
    
    # Verify the exported data
    print("\nFirst few rows of exported data:")
    pd.read_csv("MCQ_generated.csv").head()
except Exception as e:
    print(f"Error exporting to CSV: {e}")


Successfully exported quiz data to machinelearning_split_options.csv

First few rows of exported data:


In [43]:
def display_formatted_questions(df):
    """Display questions in a more readable format"""
    print("\nFormatted Quiz Questions:")
    print("=" * 80)
    
    for idx, row in df.iterrows():
        print(f"\nQuestion {idx + 1}: {row['Question']}")
        print("-" * 40)
        print(f"A) {row['Option_A']}")
        print(f"B) {row['Option_B']}")
        print(f"C) {row['Option_C']}")
        print(f"D) {row['Option_D']}")
        print(f"Correct Answer: {row['Correct_Answer']}")
        print("=" * 80)

# Display formatted questions
display_formatted_questions(quiz_df)


Formatted Quiz Questions:

Question 1: What was the primary reason for Maharana Pratap's resistance against the Mughal Empire?
----------------------------------------
A) He wanted to expand his kingdom and conquer new territories.
B) He was opposed to the Mughal Empire's religious policies.
C) He wanted to protect his homeland's independence and sovereignty.
D) He was angry with the Mughal Emperor Akbar's refusal to grant him a higher title.
Correct Answer: C

Question 2: Which battle did Maharana Pratap fight against the Mughal Empire, led by Man Singh I?
----------------------------------------
A) The Battle of Haldighati
B) The Battle of Chittor
C) The Battle of Aravalli
D) The Battle of Mewar
Correct Answer: A

Question 3: What was the name of Maharana Pratap's loyal horse that died protecting him in the Battle of Haldighati?
----------------------------------------
A) Chetak
B) Raju
C) Raja
D) Ranveer
Correct Answer: A

Question 4: How did Maharana Pratap manage to reclaim much 